In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\chinn\\OneDrive\\Desktop\\projects\\deepL\\Text-Summarizer'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [10]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directory

In [15]:
class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directory([self.config.artifacts_root])
    def get_data_transformation_config(self)->DataTransformationConfig:
         config=self.config.data_transformation
         create_directory([config.root_dir])
         data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
         return data_transformation_config

In [16]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

In [23]:
class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
    
    def convert_examples_to_features(self,example):
        input_=self.tokenizer(example['dialogue'],max_length=1024,truncation=True)

        with self.tokenizer.as_target_tokenizer():
             target_=self.tokenizer(example['summary'],max_length=128,truncation=True)

        return {
            'input_ids':input_['input_ids'],
            'attention_mask':input_['attention_mask'],
            'labels':target_['input_ids']
            }
    def convert(self):
        dataset_samsum=load_from_disk(self.config.data_path)
        dataset_samsum_pt=dataset_samsum.map(self.convert_examples_to_features,batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [25]:
try:
    config=ConfigurationManager()
    data_transformation_config=config.get_data_transformation_config()
    data_transformation=DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-12-13 23:05:00,470: INFO: common: YAML file:config\config.yaml loaded successfully]
[2024-12-13 23:05:00,474: INFO: common: YAML file:params.yaml loaded successfully]
[2024-12-13 23:05:00,477: INFO: common: Created directory: artifacts]
[2024-12-13 23:05:00,479: INFO: common: Created directory: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\chinn\anaconda3\envs\textSum\lib\site-packages\transformers\tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 54113.22 examples/s]
